# Getting started - Requirements
Python (🐍):  This assumes that you have a python environment installed. If you haven't, we recommend installing [Anaconda](https://docs.anaconda.com/anaconda/install/).

FPsim (♀️): This tutorial uses [FPsim v2.0](https://github.com/fpsim/fpsim/tree/rc2.0-methodtime) (still under development). 

To install, you first need to clone or download the a copy of the source code from https://github.com/fpsim/fpsim
 
```
git clone -b rc2.0-methodtime --single-branch https://github.com/fpsim/fpsim.git
cd fpsim
pip install -e .
```

If it worked, you should be able to import fpsim with `import fpsim as fp`.

<div class="alert alert-info">
    
An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/fpsim/fpsim/blob/main/docs/tutorials/T4_intervention_eligibility.ipynb?install=fpsim) or [Binder](https://mybinder.org/v2/gh/fpsim/fpsim/HEAD?labpath=docs%2Ftutorials%2FT4_intervention_eligibility.ipynb).
    
</div>

## Interventions - how to select a specific subpopulation (eligibilty)?

At this point, you should now be able to run multiple simulations with distinct interventions, using default or custom parameters and plot the results. So far we've seen interventions that target a model parameter. Now we will see how we can we specify a function that tells FPsim who will receive the intervention. In particular, we will modify the 'state' value (e.g., intent_to_use) of all eligible women. 


### Change the intent to use contraception of some women

In [ ]:
import fpsim as fp
import sciris as sc
import matplotlib.pyplot as plt

In [ ]:
pars = dict(
    n_agents   = 1_000,
    location   = 'kenya',
    start_year = 2000, 
    end_year   = 2012,
    exposure_factor = 1.0  # Overall scale factor on probability of becoming pregnant
)


We now define who will have their intent changed  

In [ ]:
def eligibility_criteria(sim):
    """ Select women aged 18 <= age < 25 who currently do not have intent to use contraception"""
    min_age = 18
    max_age = 25
    is_eligible = ((sim.people.is_female) &
                   (sim.people.alive)     & 
                   (sim.people.age >= min_age) &
                   (sim.people.age < max_age) &
                   (sim.people.intent_to_use == False))
    return is_eligible

In [ ]:
# This intervention will set the intent of all "eligible" women to True between the year [2005, 2008], with an annual probability "prop"
change_intent = fp.change_people_state('intent_to_use', eligibility=eligibility_criteria, years=[2005.0, 2008.0], new_val=True, prop=1.0, annual=True)

In [ ]:
# If you wanted to change the intent of a proportion of eligible population, you can pass an additional argument as below
#change_intent = fp.change_people_state('intent_to_use', eligibility=eligibility_criteria, years=[2002.0], new_val=True, prop=0.5, annual=True)

In [ ]:
s1 = fp.Sim(pars=pars, label='Baseline')
s2 = fp.Sim(pars=pars, interventions=change_intent, label='Increased intent')

simlist = sc.autolist([s1, s2])
msim = fp.MultiSim(sims=simlist)
msim.run(serial=False, compute_stats=False)

In [ ]:
msim.plot(to_plot='intent');

### Increase the number of women who have paid work

The same intervention can be used to change other 'states' or attributes of women. Let's see how we could increase the number of women who have paid work. 

In [ ]:
def select_unemployed(sim):
    """ Select women aged 15 <= age < 50 who currently do not have paid work"""
    min_age = 20
    max_age = 25
    is_eligible = ((sim.people.is_female) &
                   (sim.people.alive)     & 
                   (sim.people.age >= min_age) &
                   (sim.people.age < max_age) &
                   (sim.people.paid_employment == False))
    return is_eligible

In [ ]:
empwr = fp.Empowerment()
s3 = fp.Sim(pars=pars, empowerment_module=empwr, label='Baseline')

In [ ]:
change_employment = fp.change_people_state('paid_employment', eligibility=select_unemployed, years=2010.0, new_val=True)
empwr = fp.Empowerment()
s4 = fp.Sim(pars=pars, 
            empowerment_module=empwr, 
            interventions=change_employment, label='Increased employment')

In [ ]:
s3.run()
s4.run()

In [ ]:
plt.plot(s3.results.t, s3.results.paid_employment, label=s3.label)
plt.plot(s4.results.t, s4.results.paid_employment, label=s4.label)
plt.xlim([2005, 2012])
plt.ylabel('Percent women with paid work')
plt.xlabel('Year')
plt.legend()